In [1]:
import pandas as pd

In [11]:
from pathlib import Path

path1 = Path("../data/item_properties_part1.csv")
path2 = Path("../data/item_properties_part2.csv")

In [16]:
path = '../data/events.csv'


In [8]:
def load_item_categories(path1, path2):
    """
    Load and process item category mappings.
    
    Args:
        path1: Path to first item properties CSV
        path2: Path to second item properties CSV
    
    Returns:
        DataFrame with columns: itemid, category_id
    """
    p1 = pd.read_csv(path1)
    p2 = pd.read_csv(path2)

    items = pd.concat([p1, p2], ignore_index=True)
    cats = items[items["property"] == "categoryid"]

    cats = cats[["itemid", "value"]].drop_duplicates()
    cats = cats.rename(columns={"value": "category_id"})

    return cats

In [12]:
cat_df=load_item_categories(path1, path2)

In [14]:
def load_and_clean_events(path, cat_df, is_train=True, train_stats=None):
    """
    Load and preprocess event data with feature engineering.
    
    Args:
        path: Path to CSV file containing events
        cat_df: DataFrame with item->category mappings
        is_train: If True, compute training statistics. If False, use provided stats.
        train_stats: Dictionary of precomputed statistics (required if is_train=False)
    
    Returns:
        If is_train=True: (processed_df, train_stats)
        If is_train=False: processed_df
    """
    df = pd.read_csv(path)

    # Handle timestamp conversion properly
    if df["timestamp"].dtype == 'object':
        # Already datetime string from CSV
        df["timestamp"] = pd.to_datetime(df["timestamp"])
    else:
        # Raw milliseconds
        df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
    
    # Drop any invalid timestamps
    df = df.dropna(subset=["timestamp"])

    # Temporal features
    df["hour"] = df["timestamp"].dt.hour
    df["dayofweek"] = df["timestamp"].dt.dayofweek
    df["is_weekend"] = (df["dayofweek"] >= 5).astype(int)
    df["is_evening"] = ((df["hour"] >= 18) & (df["hour"] <= 23)).astype(int)
    df["is_working_hours"] = ((df["hour"] >= 9) & (df["hour"] <= 17)).astype(int)

    # Create target variable
    df["label"] = (df["event"] == "transaction").astype(int)

    # Merge category information
    df = df.merge(cat_df, on="itemid", how="left")
    df["category_id"] = df["category_id"].fillna(-1).astype(int)

    # Compute training statistics
    if is_train:
        train_stats = {}

        # Item popularity (total interactions per item)
        train_stats["item_popularity"] = (
            df.groupby("itemid").size().reset_index(name="item_popularity")
        )

        # Item conversion rate (purchases / total interactions)
        item_conv = df.groupby("itemid")["label"].agg(["sum", "count"]).reset_index()
        item_conv["item_conversion_rate"] = item_conv["sum"] / item_conv["count"]
        train_stats["item_conversion_rate"] = item_conv[
            ["itemid", "item_conversion_rate"]
        ]

        # Category popularity
        train_stats["category_popularity"] = (
            df.groupby("category_id").size().reset_index(name="category_popularity")
        )

        # Category conversion rate
        cat_conv = df.groupby("category_id")["label"].agg(["sum", "count"]).reset_index()
        cat_conv["category_conversion_rate"] = cat_conv["sum"] / cat_conv["count"]
        train_stats["category_conversion_rate"] = cat_conv[
            ["category_id", "category_conversion_rate"]
        ]

        # User activity level
        user_activity = (
            df.groupby("visitorid").size().reset_index(name="user_event_count")
        )
        train_stats["user_activity"] = user_activity

        # User purchase rate
        user_conv = df.groupby("visitorid")["label"].agg(["sum", "count"]).reset_index()
        user_conv["user_purchase_rate"] = user_conv["sum"] / user_conv["count"]
        train_stats["user_purchase_rate"] = user_conv[
            ["visitorid", "user_purchase_rate"]
        ]

        # Recent item popularity (last 7 days)
        max_ts = df["timestamp"].max()
        recent_cutoff = max_ts - pd.Timedelta(days=7)
        recent_df = df[df["timestamp"] >= recent_cutoff]
        train_stats["recent_item_popularity"] = (
            recent_df.groupby("itemid")
            .size()
            .reset_index(name="recent_item_popularity")
        )

        # Item popularity for ratio calculation
        train_stats["item_popularity_for_ratio"] = train_stats["item_popularity"]

        # Ensure all required stats exist (for edge cases)
        required = {
            "item_popularity": ["itemid", "item_popularity"],
            "item_conversion_rate": ["itemid", "item_conversion_rate"],
            "category_popularity": ["category_id", "category_popularity"],
            "category_conversion_rate": ["category_id", "category_conversion_rate"],
            "user_activity": ["visitorid", "user_event_count"],
            "user_purchase_rate": ["visitorid", "user_purchase_rate"],
            "recent_item_popularity": ["itemid", "recent_item_popularity"],
        }

        for key, cols in required.items():
            if key not in train_stats:
                train_stats[key] = pd.DataFrame(columns=cols)

    # Merge precomputed statistics
    df = df.merge(train_stats["item_popularity"], on="itemid", how="left")
    df["item_popularity"] = df["item_popularity"].fillna(0)

    df = df.merge(train_stats["item_conversion_rate"], on="itemid", how="left")
    df["item_conversion_rate"] = df["item_conversion_rate"].fillna(0)

    df = df.merge(train_stats["category_popularity"], on="category_id", how="left")
    df["category_popularity"] = df["category_popularity"].fillna(0)

    df = df.merge(train_stats["category_conversion_rate"], on="category_id", how="left")
    df["category_conversion_rate"] = df["category_conversion_rate"].fillna(0)

    df = df.merge(train_stats["user_activity"], on="visitorid", how="left")
    df["user_event_count"] = df["user_event_count"].fillna(0)

    df = df.merge(train_stats["user_purchase_rate"], on="visitorid", how="left")
    df["user_purchase_rate"] = df["user_purchase_rate"].fillna(0)

    # Sort by timestamp for sequential features
    df = df.sort_values("timestamp")

    # Sequential interaction features (cumulative counts)
    df["user_item_interaction_count"] = (
        df.groupby(["visitorid", "itemid"]).cumcount()
    )

    df["user_category_affinity"] = (
        df.groupby(["visitorid", "category_id"]).cumcount()
    )

    # Recent item popularity
    df = df.merge(train_stats["recent_item_popularity"], on="itemid", how="left")
    df["recent_item_popularity"] = df["recent_item_popularity"].fillna(0)

    # Item popularity ratio (how popular is this item compared to its category average?)
    category_avg_popularity = (
        train_stats["item_popularity_for_ratio"]
        .merge(df[["itemid", "category_id"]].drop_duplicates(), on="itemid")
        .groupby("category_id")["item_popularity"]
        .mean()
        .reset_index()
        .rename(columns={"item_popularity": "category_avg_item_popularity"})
    )

    df = df.merge(category_avg_popularity, on="category_id", how="left")
    df["item_popularity_ratio"] = (
        df["item_popularity"] / (df["category_avg_item_popularity"] + 1)
    ).fillna(0)

    # Select final feature set
    df = df[
        [
            "visitorid",
            "itemid",
            "category_id",
            "hour",
            "dayofweek",
            "is_weekend",
            "is_evening",
            "is_working_hours",
            "item_popularity",
            "item_conversion_rate",
            "recent_item_popularity",
            "item_popularity_ratio",
            "category_popularity",
            "category_conversion_rate",
            "user_event_count",
            "user_purchase_rate",
            "user_item_interaction_count",
            "user_category_affinity",
            "label",
            "timestamp",
        ]
    ]

    if is_train:
        return df, train_stats
    else:
        return df


In [26]:
train_df, train_stats = load_and_clean_events(
    path,
    cat_df,
    is_train=True
)


In [28]:
train_df.to_csv('abc.csv')

In [29]:
train_df

,visitorid,itemid,category_id,hour,dayofweek,is_weekend,is_evening,is_working_hours,item_popularity,item_conversion_rate,recent_item_popularity,item_popularity_ratio,category_popularity,category_conversion_rate,user_event_count,user_purchase_rate,user_item_interaction_count,user_category_affinity,label,timestamp
0,693516,297662,1130,3,6,1,0,0,63,0.000000,4.0,3.917647,1116,0.008065,3,0.0,0,0,0,2015-05-03 03:00:04.384
1,829044,60987,463,3,6,1,0,0,79,0.000000,2.0,2.692308,3938,0.000000,1,0.0,0,0,0,2015-05-03 03:00:11.289
2,652699,252860,-1,3,6,1,0,0,248,0.000000,12.0,40.452259,255585,0.001858,1,0.0,0,0,0,2015-05-03 03:00:13.048
3,1125936,33661,1628,3,6,1,0,0,63,0.000000,1.0,6.450377,1429,0.004199,1,0.0,0,0,0,2015-05-03 03:00:24.154
4,693516,297662,1130,3,6,1,0,0,63,0.000000,4.0,3.917647,1116,0.008065,3,0.0,1,1,0,2015-05-03 03:00:26.228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2965026,472345,301436,1244,2,4,0,0,0,2,0.000000,2.0,0.189713,10656,0.011167,2,0.0,1,0,0,2015-09-18 02:59:24.029
2965027,1207677,310922,565,2,4,0,0,0,27,0.000000,2.0,1.943204,4281,0.002569,1,0.0,0,0,0,2015-09-18 02:59:25.499
2965028,255126,47467,491,2,4,0,0,0,4,0.000000,2.0,0.078016,61114,0.004303,1,0.0,0,0,0,2015-09-18 02:59:34.109
2965029,622226,345308,427,2,4,0,0,0,132,0.015152,8.0,4.721386,5122,0.019719,1,0.0,0,0,0,2015-09-18 02:59:41.778


In [31]:
train_df[train_df['visitorid']== 693516]

,visitorid,itemid,category_id,hour,dayofweek,is_weekend,is_evening,is_working_hours,item_popularity,item_conversion_rate,recent_item_popularity,item_popularity_ratio,category_popularity,category_conversion_rate,user_event_count,user_purchase_rate,user_item_interaction_count,user_category_affinity,label,timestamp
0,693516,297662,1130,3,6,1,0,0,63,0.0,4.0,3.917647,1116,0.008065,3,0.0,0,0,0,2015-05-03 03:00:04.384
4,693516,297662,1130,3,6,1,0,0,63,0.0,4.0,3.917647,1116,0.008065,3,0.0,1,1,0,2015-05-03 03:00:26.228
8,693516,297662,1130,3,6,1,0,0,63,0.0,4.0,3.917647,1116,0.008065,3,0.0,2,2,0,2015-05-03 03:00:29.427


In [33]:
train_df['visitorid'].value_counts().sort_values(ascending=False)

visitorid
1150086    8148
530559     4497
152963     3217
895999     2618
163561     2592
           ... 
1207677       1
624573        1
998517        1
525390        1
485144        1
Name: count, Length: 1407580, dtype: int64